In [43]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import Library

In [44]:
df1 = pd.read_csv("../input/university-salaries/university-salaries/salaries_final.csv")
df2 = pd.read_csv("../input/university-salaries/university-salaries/salaries_without_dept.csv")
df3 = pd.read_csv("../input/university-salaries/university-salaries/data_dictionary.csv")

แสดงข้อมูลขั้นต้น

In [45]:
df1

In [46]:
df1_year = df1.groupby(['Year'])
df1_year.nunique()

In [47]:
df1_name = df1.groupby(['Name'])
df1_name.nunique()

In [48]:
df1_job = df1.groupby(['Primary Job Title'])
df1_job.nunique()

In [49]:
df1_dept = df1.groupby(['Department'])
df1_dept.nunique()

In [50]:
df1_coll = df1.groupby(['College'])
df1_coll.nunique()

เปรียบเทียบจำนวนบุคลากรแต่ละตำแหน่ง

In [51]:
df1['Primary Job Title'].value_counts().head(20)

ดูการเพิ่มขึ้นของตำแหน่งงาน 5 อันดับแรก

In [52]:
df1_job = df1.loc[df1['Primary Job Title'].isin(['Assistant Professor', 'Associate Professor', 'Professor', 'Lecturer', 'Senior Lecturer'])]
fig, ax = plt.subplots(1, 5, figsize = (20, 5), sharex = True)
job = ['Assistant Professor', 'Associate Professor', 'Professor', 'Lecturer', 'Senior Lecturer']
for i in range(0,5):
    plt.sca(ax[i])
    df1_job.loc[df1_job['Primary Job Title'] == job[i], 'Year'].value_counts().sort_values().plot(kind = 'bar')
    plt.title(job[i], fontsize = 16)
    plt.xlabel("Years")
plt.tight_layout
plt.show()

เปรียบเทียบเงินเดือนของตำแหน่งข้างต้น

In [53]:
plt.figure(figsize=(16,5))
ax = plt.gca()
df1_job.groupby(["Primary Job Title", "Year"])["Base Pay"].mean().unstack(level=0).plot(kind="line", ax=ax)
plt.ylabel("Mean base pay among all faculty")
plt.show()

In [67]:
# subset data 
df_pay = df1_job.loc[df1_job["Year"]==2020, "Base Pay"]

# display quantiles in a dataframe
q = [0, .25, .50, .75, 1]
display(pd.DataFrame({ "Quantile":q, "Value":np.quantile(df_pay.values, q) }))

# print mean
print("Mean:", df_pay.mean())

# plot distribution
df_pay.plot(kind="hist", bins=30, edgecolor="black")
plt.show()

เปรียบเทียบเงินเดือนเฉลี่ยแต่ละปีของ college

In [54]:
df1.groupby(['College', 'Year'])[['Base Pay']].mean()

In [55]:
coll = df3['College Abbreviation']
for i in range(0,12):
    df1_coll_year = df1.loc[df1['College'] == coll[i]].groupby(['College', 'Year'])[['Base Pay']].mean().sort_values(by = 'Year').plot(kind = 'bar')
    plt.title(coll[i], fontsize = 16)
    plt.ylabel("Average Base Pay")
plt.tight_layout
plt.show()

สรุปผลจากกราฟข้างต้น

In [56]:
plt.figure(figsize=(20,5))
ax = plt.gca()
df1.groupby(["College", "Year"])["Base Pay"].mean().unstack(level=0).plot(kind="line", ax = ax)
plt.ylabel("Median base pay among all college")
plt.legend(loc="center right")
plt.show()

In [57]:
plt.figure(figsize=(20,5))
ax = plt.gca()
df1.groupby(["College", "Year"])["Base Pay"].mean().unstack(level=1).plot(kind="bar", ax=ax)
plt.ylabel("Mean base pay among all faculty")
plt.legend(loc = 'right')
plt.show()

เปรียบเทียบเงินเดือนของ college โดยแบ่งเป็นแต่ละ department 

In [58]:
dept = df1['Department'].unique()
coll = df3['College Abbreviation']
for i in range(0,12):
    df1_dept_coll = df1.loc[df1['College'] == coll[i]].groupby(['College', 'Department'])[['Base Pay']].mean().sort_values(by = 'Base Pay').head(5).plot(kind = 'barh')
    plt.title(coll[i], fontsize = 16)
    plt.xlabel("Average Base Pay")
plt.tight_layout
plt.show()

เปรียบเทียบจำนวนจำนวน college ทุก 3 ปี

In [59]:
fig, ax = plt.subplots(1, 4, figsize=(12,5), sharex = True)
year = [2010, 2014, 2017, 2020]
for i in range(4):
    plt.sca(ax[i])
    df1.loc[df1["Year"]==year[i], "College"].value_counts(normalize=True).plot(kind="bar")
    plt.xlabel("Fraction of faculty belonging to college", fontsize=7)
    plt.title(year[i], fontsize=16)
    
plt.show()

In [61]:
x = [x for x in df1['Primary Job Title'].unique() if 'lecturer' in x.lower()]
df1.loc[df1['Primary Job Title'].isin(x)].groupby('Primary Job Title')[['Base Pay']].mean()

In [62]:
colleges_under = df3.loc[df3['Undergraduate College'] == 'Yes', 'College Abbreviation']
colleges_under